# A concrete example of sparse frequency in a TFIM model with HVA circuit

In [204]:
import numpy as np
from itertools import product

# Define single-qubit |+> and |-> states
ket_plus = np.array([1, 1], dtype=complex) / np.sqrt(2)
ket_minus = np.array([1, -1], dtype=complex) / np.sqrt(2)


def generate_all_pm_states(n):
    """
    Generate all n-qubit |±> tensor-product states.
    Returns a dict: key is label string like '++-', value is the 2^n column vector.
    """
    states = {}
    for bits in product(['+', '-'], repeat=n):
        label = ''.join(bits)
        vec = np.array([1], dtype=complex)
        for b in bits:
            vec = np.kron(vec, ket_plus if b == '+' else ket_minus)
        states[label] = vec
    return states

def verify_orthonormality(states):
    """
    Given a dict of label->state vectors, verify they form an orthonormal set.
    Prints the maximum deviation from orthonormality.
    """
    labels = list(states.keys())
    N = len(labels)
    # Assemble matrix of inner products
    gram = np.zeros((N, N), dtype=complex)
    for i, li in enumerate(labels):
        for j, lj in enumerate(labels):
            gram[i, j] = np.vdot(states[li], states[lj])
    
    # Ideal identity
    identity = np.eye(N, dtype=complex)
    deviation = gram - identity
    max_offdiag = np.max(np.abs(deviation - np.diag(np.diag(deviation))))
    max_diag_deviation = np.max(np.abs(np.diag(deviation)))
    
    print(f"Max off-diagonal absolute overlap: {max_offdiag:.2e}")
    print(f"Max diagonal deviation from 1: {max_diag_deviation:.2e}")
    if np.allclose(gram, identity, atol=1e-8):
        print("Verification: PASS — states are orthonormal.")
    else:
        print("Verification: FAIL — states are not perfectly orthonormal.")

# Example: verify for n = 3 qubits
n = 3
states = generate_all_pm_states(n)
verify_orthonormality(states)

Max off-diagonal absolute overlap: 2.36e-17
Max diagonal deviation from 1: 4.44e-16
Verification: PASS — states are orthonormal.


In [205]:
# 示例运行
n = 3
pm_states = generate_all_pm_states(n)
print(f"{n} qubits |±> 态，共 {len(pm_states)} 种：", list(pm_states.keys()))
pm_states['+++']

3 qubits |±> 态，共 8 种： ['+++', '++-', '+-+', '+--', '-++', '-+-', '--+', '---']


array([0.354+0.j, 0.354+0.j, 0.354+0.j, 0.354+0.j, 0.354+0.j, 0.354+0.j,
       0.354+0.j, 0.354+0.j])

In [206]:
# 定义 Pauli 矩阵
paulis = {
    'I': np.eye(2, dtype=complex),
    'X': np.array([[0, 1], [1, 0]], dtype=complex),
    'Y': np.array([[0, -1j], [1j, 0]], dtype=complex),
    'Z': np.array([[1, 0], [0, -1]], dtype=complex),
}

def pauli_word(word):
    """
    给定一个 Pauli 字符串（如 'XZI'），返回对应的 2^n x 2^n 矩阵。
    """
    mat = np.array([1], dtype=complex)
    for c in word:
        mat = np.kron(mat, paulis[c])
    return mat

sample_word = 'ZZI'
pw_matrix = pauli_word(sample_word)
print(f"\nPauli word '{sample_word}' 的矩阵：\n", pw_matrix)



Pauli word 'ZZI' 的矩阵：
 [[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j -0.+0.j  0.+0.j  0.+0.j -0.+0.j -0.+0.j]
 [ 0.+0.j  0.+0.j -0.+0.j -1.+0.j  0.+0.j  0.+0.j -0.+0.j -0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.+0.j -0.+0.j -0.+0.j -0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+0.j -1.+0.j -0.+0.j -0.+0.j]
 [ 0.+0.j  0.+0.j -0.+0.j -0.+0.j -0.+0.j -0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -0.+0.j -0.+0.j -0.+0.j -0.+0.j  0.+0.j  1.+0.j]]


In [207]:
# Function to get matrix representation in the |±> basis
def pm_basis_matrix(M, n):
    """
    Given a 2^n x 2^n matrix M and qubit count n,
    returns (labels, M_pm) where labels is the list of basis labels
    (sorted lexicographically) and M_pm is the matrix in the |±> basis.
    """
    states = generate_all_pm_states(n)
    labels = sorted(states.keys())
    
    # Construct change-of-basis matrix U whose columns are the |±> states
    U = np.column_stack([states[label] for label in labels])
    
    # Compute the representation in the new basis: U† M U
    M_pm = U.conj().T @ M @ U
    return labels, M_pm

M_example = pauli_word('ZZI')+pauli_word('IZZ')+pauli_word('ZIZ')

labels, M_pm = pm_basis_matrix(M_example, n=3)
print(labels)

np.set_printoptions(precision=3, suppress=True)
print(M_pm)

['+++', '++-', '+-+', '+--', '-++', '-+-', '--+', '---']
[[-0.+0.j -0.+0.j -0.+0.j  1.+0.j -0.+0.j  1.+0.j  1.+0.j -0.+0.j]
 [-0.+0.j -0.+0.j  1.+0.j -0.+0.j  1.+0.j -0.+0.j -0.+0.j  1.+0.j]
 [-0.+0.j  1.+0.j -0.+0.j -0.+0.j  1.+0.j -0.+0.j -0.+0.j  1.+0.j]
 [ 1.+0.j -0.+0.j -0.+0.j -0.+0.j -0.+0.j  1.+0.j  1.+0.j -0.+0.j]
 [-0.+0.j  1.+0.j  1.+0.j -0.+0.j -0.+0.j -0.+0.j -0.+0.j  1.+0.j]
 [ 1.+0.j -0.+0.j -0.+0.j  1.+0.j -0.+0.j -0.+0.j  1.+0.j -0.+0.j]
 [ 1.+0.j -0.+0.j -0.+0.j  1.+0.j -0.+0.j  1.+0.j -0.+0.j -0.+0.j]
 [-0.+0.j  1.+0.j  1.+0.j -0.+0.j  1.+0.j -0.+0.j -0.+0.j -0.+0.j]]


In [208]:
import numpy as np
from scipy.linalg import expm

def generate_hamiltonians(n):
    """
    Generate transverse-field Ising model Hamiltonians for n qubits:
        Hxz = sum_i Z_i Z_{i+1}  (periodic boundary)
        Hx  = sum_i X_i
    Returns:
        Hzz, Hx   (each is a 2^n × 2^n numpy array)
    """
    # Pauli matrices
    X = np.array([[0, 1], [1, 0]], dtype=complex)
    Z = np.array([[1, 0], [0, -1]], dtype=complex)
    I = np.eye(2, dtype=complex)
    
    def kron_site(op, site):
        """Kron product placing `op` at position `site`, I elsewhere."""
        mats = [op if j == site else I for j in range(n)]
        result = mats[0]
        for m in mats[1:]:
            result = np.kron(result, m)
        return result

    # Build Hx = sum_i X_i
    Hx = sum(kron_site(X, i) for i in range(n))
    # Build Hzz = sum_i Z_i Z_{i+1} with periodic boundary
    Hzz = sum(kron_site(Z, i) @ kron_site(Z, (i + 1) % n) for i in range(n))
    return Hzz, Hx

def expm_hermitian(H, theta):
    """
    Compute U = exp(-i * theta/2 * H) for Hermitian H
    """
    return expm(-1j * (theta / 2) * H)

def build_hva_circuit(n, layers, thetas):
    """
    Build the HVA circuit unitary U(theta) for TFIM with n qubits and given layers.
    Args:
      n      : number of qubits
      layers : number of alternating (Hzz, Hx) layers
      thetas : list or array of length 2*layers, angles [θ1, θ2, ..., θ_{2L}]
    Returns:
      2^n × 2^n unitary matrix U
    """
    if len(thetas) != 2 * layers:
        raise ValueError("Length of theta list must be 2 * layers.")
    
    # Hamiltonians
    Hzz, Hx = generate_hamiltonians(n)
    
    # Start from identity
    U = np.eye(2**n, dtype=complex)
    
    # Apply layers: for each layer j,
    #   U = exp(-i θ_{2j}/2 * Hx) · exp(-i θ_{2j-1}/2 * Hzz) · U
    for j in range(layers):
        theta_zz = thetas[2*j]
        theta_x  = thetas[2*j + 1]
        U = expm_hermitian(Hx, theta_x) @ expm_hermitian(Hzz, theta_zz) @ U
    
    return U


n = 3
layers = 2
thetas = np.random.random(2*layers)
U = build_hva_circuit(n, layers, thetas)


In [209]:
M_example = pauli_word('ZZI')+pauli_word('IZZ')+pauli_word('ZIZ') + 0.5*pauli_word('XII')+pauli_word('IXI')+pauli_word('IIX')
# M_example = U.conj().T @ M_example @ U
labels, M_pm = pm_basis_matrix(M_example, n=3)
print(labels)
print(M_pm)

['+++', '++-', '+-+', '+--', '-++', '-+-', '--+', '---']
[[ 2.5+0.j  0. +0.j  0. +0.j  1. +0.j  0. +0.j  1. +0.j  1. +0.j  0. +0.j]
 [-0. +0.j  0.5+0.j  1. +0.j -0. +0.j  1. +0.j -0. +0.j -0. +0.j  1. +0.j]
 [-0. +0.j  1. +0.j  0.5+0.j -0. +0.j  1. +0.j -0. +0.j -0. +0.j  1. +0.j]
 [ 1. +0.j -0. +0.j -0. +0.j -1.5+0.j -0. +0.j  1. +0.j  1. +0.j  0. +0.j]
 [ 0. +0.j  1. +0.j  1. +0.j  0. +0.j  1.5+0.j  0. +0.j  0. +0.j  1. +0.j]
 [ 1. +0.j -0. +0.j  0. +0.j  1. +0.j -0. +0.j -0.5+0.j  1. +0.j -0. +0.j]
 [ 1. +0.j  0. +0.j -0. +0.j  1. +0.j -0. +0.j  1. +0.j -0.5+0.j -0. +0.j]
 [-0. +0.j  1. +0.j  1. +0.j  0. +0.j  1. +0.j -0. +0.j -0. +0.j -2.5+0.j]]
